# LR

In [105]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer

import numpy as np
import pandas as pd


In [2]:
train_data = spark.read.text('C:\spark\data\my_data\iris_data.txt')
train_data.head(5)
# type(train_data)


[Row(value='5.1,3.5,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.0,1.4,0.2,Iris-setosa'),
 Row(value='4.7,3.2,1.3,0.2,Iris-setosa'),
 Row(value='4.6,3.1,1.5,0.2,Iris-setosa'),
 Row(value='5.0,3.6,1.4,0.2,Iris-setosa')]

In [3]:
train_data.collect()

[Row(value='5.1,3.5,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.0,1.4,0.2,Iris-setosa'),
 Row(value='4.7,3.2,1.3,0.2,Iris-setosa'),
 Row(value='4.6,3.1,1.5,0.2,Iris-setosa'),
 Row(value='5.0,3.6,1.4,0.2,Iris-setosa'),
 Row(value='5.4,3.9,1.7,0.4,Iris-setosa'),
 Row(value='4.6,3.4,1.4,0.3,Iris-setosa'),
 Row(value='5.0,3.4,1.5,0.2,Iris-setosa'),
 Row(value='4.4,2.9,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.1,1.5,0.1,Iris-setosa'),
 Row(value='5.4,3.7,1.5,0.2,Iris-setosa'),
 Row(value='4.8,3.4,1.6,0.2,Iris-setosa'),
 Row(value='4.8,3.0,1.4,0.1,Iris-setosa'),
 Row(value='4.3,3.0,1.1,0.1,Iris-setosa'),
 Row(value='5.8,4.0,1.2,0.2,Iris-setosa'),
 Row(value='5.7,4.4,1.5,0.4,Iris-setosa'),
 Row(value='5.4,3.9,1.3,0.4,Iris-setosa'),
 Row(value='5.1,3.5,1.4,0.3,Iris-setosa'),
 Row(value='5.7,3.8,1.7,0.3,Iris-setosa'),
 Row(value='5.1,3.8,1.5,0.3,Iris-setosa'),
 Row(value='5.4,3.4,1.7,0.2,Iris-setosa'),
 Row(value='5.1,3.7,1.5,0.4,Iris-setosa'),
 Row(value='4.6,3.6,1.0,0.2,Iris-setosa'),
 Row(value=

In [4]:
train_data = spark.read.csv('C:\spark\data\my_data\iris_data.csv', header=False)
train_data.head(5)

[Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='1.0'),
 Row(_c0='4.9', _c1='3.0', _c2='1.4', _c3='0.2', _c4='1.0'),
 Row(_c0='4.7', _c1='3.2', _c2='1.3', _c3='0.2', _c4='1.0'),
 Row(_c0='4.6', _c1='3.1', _c2='1.5', _c3='0.2', _c4='1.0'),
 Row(_c0='5.0', _c1='3.6', _c2='1.4', _c3='0.2', _c4='1.0')]

In [5]:
train_data.show(5)

+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|
+---+---+---+---+---+
|5.1|3.5|1.4|0.2|1.0|
|4.9|3.0|1.4|0.2|1.0|
|4.7|3.2|1.3|0.2|1.0|
|4.6|3.1|1.5|0.2|1.0|
|5.0|3.6|1.4|0.2|1.0|
+---+---+---+---+---+
only showing top 5 rows



In [6]:
train_data.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [7]:
train_data.select('_c0').show(10)

+---+
|_c0|
+---+
|5.1|
|4.9|
|4.7|
|4.6|
|5.0|
|5.4|
|4.6|
|5.0|
|4.4|
|4.9|
+---+
only showing top 10 rows



In [8]:

train_data = train_data.withColumn('_c0', train_data['_c0'].cast('double'))\
       .withColumn('_c1', train_data['_c1'].cast('double'))\
       .withColumn('_c2', train_data['_c2'].cast('double'))\
       .withColumn('_c3', train_data['_c3'].cast('double'))
input_col = ['_c0', '_c1', '_c2', '_c3']
vecAssembler = VectorAssembler(inputCols=input_col, outputCol="features")
new_train_data = vecAssembler.transform(train_data)
# stringIndexer = StringIndexer(inputCol="_c4", outputCol="label")
# pipeline = Pipeline(stages=[vecAssembler, stringIndexer])
# pipelineFit = pipeline.fit(train_data)
# dataset = pipelineFit.transform(train_data)

In [9]:
new_train_data

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector]

In [112]:
new_train_data.show(500)

+---+---+---+---+---+-----------------+
|_c0|_c1|_c2|_c3|_c4|         features|
+---+---+---+---+---+-----------------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|
|5.0|3.4|1.5|0.2|1.0|[5.0,3.4,1.5,0.2]|
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|
|4.9|3.1|1.5|0.1|1.0|[4.9,3.1,1.5,0.1]|
|5.4|3.7|1.5|0.2|1.0|[5.4,3.7,1.5,0.2]|
|4.8|3.4|1.6|0.2|1.0|[4.8,3.4,1.6,0.2]|
|4.8|3.0|1.4|0.1|1.0|[4.8,3.0,1.4,0.1]|
|4.3|3.0|1.1|0.1|1.0|[4.3,3.0,1.1,0.1]|
|5.8|4.0|1.2|0.2|1.0|[5.8,4.0,1.2,0.2]|
|5.7|4.4|1.5|0.4|1.0|[5.7,4.4,1.5,0.4]|
|5.4|3.9|1.3|0.4|1.0|[5.4,3.9,1.3,0.4]|
|5.1|3.5|1.4|0.3|1.0|[5.1,3.5,1.4,0.3]|
|5.7|3.8|1.7|0.3|1.0|[5.7,3.8,1.7,0.3]|
|5.1|3.8|1.5|0.3|1.0|[5.1,3.8,1.5,0.3]|
|5.4|3.4|1.7|0.2|1.0|[5.4,3.4,1.7,0.2]|
|5.1|3.7|1.5|0.4|1.0|[5.1,3.7,1.5,0.4]|


In [23]:
new_new_train_data = new_train_data.withColumn('label', new_train_data['_c4'].cast('double'))

In [24]:
new_new_train_data

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double]

In [25]:
new_new_train_data.show(5)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  1.0|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  1.0|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  1.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  1.0|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  1.0|
+---+---+---+---+---+-----------------+-----+
only showing top 5 rows



In [26]:
train_data.show(5)

+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|
+---+---+---+---+---+
|5.1|3.5|1.4|0.2|1.0|
|4.9|3.0|1.4|0.2|1.0|
|4.7|3.2|1.3|0.2|1.0|
|4.6|3.1|1.5|0.2|1.0|
|5.0|3.6|1.4|0.2|1.0|
+---+---+---+---+---+
only showing top 5 rows



In [28]:
# 新_模型训练
lr_3 = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0)
lrModel_3 = lr_3.fit(new_new_train_data)

In [30]:
pre_3 = lrModel_3.transform(new_new_train_data)

In [113]:
pre_3.show(500)

+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|_c0|_c1|_c2|_c3|_c4|         features|label|       rawPrediction|         probability|prediction|
+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  1.0|[-3.5921473185105...|[0.00157155276833...|       1.0|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  1.0|[-3.5335755527481...|[0.00215601550463...|       1.0|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  1.0|[-3.5417261943304...|[0.00184854030564...|       1.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  1.0|[-3.5284115349670...|[0.00201701076712...|       1.0|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  1.0|[-3.5966005410309...|[0.00144326216378...|       1.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  1.0|[-3.6511035819565...|[0.00135870845931...|       1.0|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  1.0|[-3.5578601196240...|[0.00167224197126...|       1.0|
|5.0|3.4|1

In [111]:
prob_3 = pre_3.select('probability')
prob_3.collect()

[Row(probability=DenseVector([0.0016, 0.7914, 0.1504, 0.0567])),
 Row(probability=DenseVector([0.0022, 0.6804, 0.2289, 0.0885])),
 Row(probability=DenseVector([0.0018, 0.7571, 0.1722, 0.0689])),
 Row(probability=DenseVector([0.002, 0.7275, 0.1909, 0.0796])),
 Row(probability=DenseVector([0.0014, 0.8187, 0.1297, 0.0501])),
 Row(probability=DenseVector([0.0014, 0.8193, 0.1258, 0.0536])),
 Row(probability=DenseVector([0.0017, 0.7935, 0.1418, 0.063])),
 Row(probability=DenseVector([0.0017, 0.7704, 0.1641, 0.0638])),
 Row(probability=DenseVector([0.0022, 0.6971, 0.2109, 0.0899])),
 Row(probability=DenseVector([0.002, 0.7149, 0.207, 0.0761])),
 Row(probability=DenseVector([0.0014, 0.8083, 0.1398, 0.0506])),
 Row(probability=DenseVector([0.0017, 0.7795, 0.1552, 0.0636])),
 Row(probability=DenseVector([0.0021, 0.7029, 0.2154, 0.0796])),
 Row(probability=DenseVector([0.0019, 0.763, 0.1688, 0.0664])),
 Row(probability=DenseVector([0.0011, 0.8533, 0.1099, 0.0357])),
 Row(probability=DenseVector([

In [44]:
prob_3 = prob_3.collect()

In [66]:
new_prob_3 = pre_3.select(['_c0', '_c1', 'probability'])

In [115]:
new_prob_3.show(5)

+---+---+--------------------+
|_c0|_c1|         probability|
+---+---+--------------------+
|5.1|3.5|[0.00157155276833...|
|4.9|3.0|[0.00215601550463...|
|4.7|3.2|[0.00184854030564...|
|4.6|3.1|[0.00201701076712...|
|5.0|3.6|[0.00144326216378...|
+---+---+--------------------+
only showing top 5 rows



In [101]:
new_prob_3

DataFrame[_c0: double, _c1: double, probability: vector]

In [104]:
new_prob_3.probability

Column<b'probability'>

In [131]:
vs = VectorSlicer(inputCol="probability", outputCol="prob_1", indices=[1])
prob_1 = vs.transform(new_prob_3)
prob_1.show(5)



+---+---+--------------------+--------------------+
|_c0|_c1|         probability|              prob_1|
+---+---+--------------------+--------------------+
|5.1|3.5|[0.00157155276833...|[0.7913557718676799]|
|4.9|3.0|[0.00215601550463...|[0.6804221441922835]|
|4.7|3.2|[0.00184854030564...|[0.7571145009018414]|
|4.6|3.1|[0.00201701076712...|[0.7274824434397575]|
|5.0|3.6|[0.00144326216378...|[0.8187212689762617]|
+---+---+--------------------+--------------------+
only showing top 5 rows



In [127]:
new_prob_3 = new_prob_3.withColumn('prob_1', prob_1)

AnalysisException: 'Resolved attribute(s) prob_1#875 missing from _c0#60,_c1#66,probability#431 in operator !Project [_c0#60, _c1#66, probability#431, prob_1#875 AS prob_1#880].;;\n!Project [_c0#60, _c1#66, probability#431, prob_1#875 AS prob_1#880]\n+- AnalysisBarrier\n      +- Project [_c0#60, _c1#66, probability#431]\n         +- Project [_c0#60, _c1#66, _c2#72, _c3#78, _c4#19, features#84, label#228, rawPrediction#422, probability#431, UDF(rawPrediction#422) AS prediction#441]\n            +- Project [_c0#60, _c1#66, _c2#72, _c3#78, _c4#19, features#84, label#228, rawPrediction#422, UDF(rawPrediction#422) AS probability#431]\n               +- Project [_c0#60, _c1#66, _c2#72, _c3#78, _c4#19, features#84, label#228, UDF(features#84) AS rawPrediction#422]\n                  +- Project [_c0#60, _c1#66, _c2#72, _c3#78, _c4#19, features#84, cast(_c4#19 as double) AS label#228]\n                     +- Project [_c0#60, _c1#66, _c2#72, _c3#78, _c4#19, UDF(named_struct(_c0, _c0#60, _c1, _c

In [114]:
prob_3

DataFrame[probability: vector]

In [90]:
new_prob_3 = new_prob_3.withColumn('probability', new_prob_3.probability.getItem(0))

IllegalArgumentException: "requirement failed: The number of columns doesn't match.\nOld column names (3): _c0, _c1, probability\nNew column names (0): "

In [54]:
prob_df = new_prob_3.select('probability')

In [71]:
prob_c2 = pre_3.select('_c2')
prob_c2.show(5)

+---+
|_c2|
+---+
|1.4|
|1.4|
|1.3|
|1.5|
|1.4|
+---+
only showing top 5 rows



In [74]:
new_prob_3 = new_prob_3.withColumn('new_c2', prob_c2.getItem(0))

AttributeError: 'DataFrame' object has no attribute 'getItem'

In [55]:
prob_df.show(5)

+--------------------+
|         probability|
+--------------------+
|[0.00157155276833...|
|[0.00215601550463...|
|[0.00184854030564...|
|[0.00201701076712...|
|[0.00144326216378...|
+--------------------+
only showing top 5 rows



In [58]:
prob_df[0][0]

Column<b'probability[0]'>

In [75]:
new_prob_3.withColumn('probability', prob_df.getItem(0))

AttributeError: 'DataFrame' object has no attribute 'getItem'

In [15]:
tra_df = train_data.select(['_c0', '_c1'])
tra_df

DataFrame[_c0: double, _c1: double]

In [16]:
dataset.show(10)

NameError: name 'dataset' is not defined

In [14]:
dataset.select('_c0')

DataFrame[_c0: double]

In [15]:
dataset

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double]

In [99]:
dataset.select('label')

DataFrame[label: double]

In [16]:
c_1 = dataset['_c0']
c_1

Column<b'_c0'>

In [17]:
col_v1 = dataset._c0
col_v1

Column<b'_c0'>

In [18]:
col_v2 = dataset._c1
col_v2

Column<b'_c1'>

In [19]:
col_v12 = col_v1 * col_v2
col_v12

Column<b'(_c0 * _c1)'>

In [20]:
new_dataset = dataset.withColumn('col_v12', col_v12)

In [21]:
new_dataset.show(10)

+---+---+---+---+---+-----------------+-----+------------------+
|_c0|_c1|_c2|_c3|_c4|         features|label|           col_v12|
+---+---+---+---+---+-----------------+-----+------------------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  0.0|17.849999999999998|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  0.0|14.700000000000001|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  0.0|15.040000000000001|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0|             14.26|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  0.0|              18.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  0.0|21.060000000000002|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0|15.639999999999999|
|5.0|3.4|1.5|0.2|1.0|[5.0,3.4,1.5,0.2]|  0.0|              17.0|
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|             12.76|
|4.9|3.1|1.5|0.1|1.0|[4.9,3.1,1.5,0.1]|  0.0|15.190000000000001|
+---+---+---+---+---+-----------------+-----+------------------+
only showing top 10 rows



In [22]:
col1 = dataset.select('_c0')
col1_pd = col1.toPandas()
col1_ar = col1_pd.values
col1_ar

array([[5.1],
       [4.9],
       [4.7],
       [4.6],
       [5. ],
       [5.4],
       [4.6],
       [5. ],
       [4.4],
       [4.9],
       [5.4],
       [4.8],
       [4.8],
       [4.3],
       [5.8],
       [5.7],
       [5.4],
       [5.1],
       [5.7],
       [5.1],
       [5.4],
       [5.1],
       [4.6],
       [5.1],
       [4.8],
       [5. ],
       [5. ],
       [5.2],
       [5.2],
       [4.7],
       [4.8],
       [5.4],
       [5.2],
       [5.5],
       [4.9],
       [5. ],
       [5.5],
       [4.9],
       [4.4],
       [5.1],
       [5. ],
       [4.5],
       [4.4],
       [5. ],
       [5.1],
       [4.8],
       [5.1],
       [4.6],
       [5.3],
       [5. ],
       [7. ],
       [6.4],
       [6.9],
       [5.5],
       [6.5],
       [5.7],
       [6.3],
       [4.9],
       [6.6],
       [5.2],
       [5. ],
       [5.9],
       [6. ],
       [6.1],
       [5.6],
       [6.7],
       [5.6],
       [5.8],
       [6.2],
       [5.6],
       [5.9],
      

In [23]:
col2 = dataset.select('_c1')
col2_pd = col2.toPandas()
col2_ar = col2_pd.values
col2_ar

array([[3.5],
       [3. ],
       [3.2],
       [3.1],
       [3.6],
       [3.9],
       [3.4],
       [3.4],
       [2.9],
       [3.1],
       [3.7],
       [3.4],
       [3. ],
       [3. ],
       [4. ],
       [4.4],
       [3.9],
       [3.5],
       [3.8],
       [3.8],
       [3.4],
       [3.7],
       [3.6],
       [3.3],
       [3.4],
       [3. ],
       [3.4],
       [3.5],
       [3.4],
       [3.2],
       [3.1],
       [3.4],
       [4.1],
       [4.2],
       [3.1],
       [3.2],
       [3.5],
       [3.1],
       [3. ],
       [3.4],
       [3.5],
       [2.3],
       [3.2],
       [3.5],
       [3.8],
       [3. ],
       [3.8],
       [3.2],
       [3.7],
       [3.3],
       [3.2],
       [3.2],
       [3.1],
       [2.3],
       [2.8],
       [2.8],
       [3.3],
       [2.4],
       [2.9],
       [2.7],
       [2. ],
       [3. ],
       [2.2],
       [2.9],
       [2.9],
       [3.1],
       [3. ],
       [2.7],
       [2.2],
       [2.5],
       [3.2],
      

In [24]:
coll = col1_ar * col2_ar
coll

array([[17.85],
       [14.7 ],
       [15.04],
       [14.26],
       [18.  ],
       [21.06],
       [15.64],
       [17.  ],
       [12.76],
       [15.19],
       [19.98],
       [16.32],
       [14.4 ],
       [12.9 ],
       [23.2 ],
       [25.08],
       [21.06],
       [17.85],
       [21.66],
       [19.38],
       [18.36],
       [18.87],
       [16.56],
       [16.83],
       [16.32],
       [15.  ],
       [17.  ],
       [18.2 ],
       [17.68],
       [15.04],
       [14.88],
       [18.36],
       [21.32],
       [23.1 ],
       [15.19],
       [16.  ],
       [19.25],
       [15.19],
       [13.2 ],
       [17.34],
       [17.5 ],
       [10.35],
       [14.08],
       [17.5 ],
       [19.38],
       [14.4 ],
       [19.38],
       [14.72],
       [19.61],
       [16.5 ],
       [22.4 ],
       [20.48],
       [21.39],
       [12.65],
       [18.2 ],
       [15.96],
       [20.79],
       [11.76],
       [19.14],
       [14.04],
       [10.  ],
       [17.7 ],
       [

In [25]:
coll_df = pd.DataFrame(coll)

In [26]:
type(coll_df)

pandas.core.frame.DataFrame

In [27]:
coll_spark_df = spark.createDataFrame(coll_df)

In [28]:
coll_spark_df

DataFrame[0: double]

In [29]:
type(coll_spark_df)

pyspark.sql.dataframe.DataFrame

In [30]:
coll_cl = coll_spark_df.collect()

In [31]:
coll_cl

[Row(0=17.849999999999998),
 Row(0=14.700000000000001),
 Row(0=15.040000000000001),
 Row(0=14.26),
 Row(0=18.0),
 Row(0=21.060000000000002),
 Row(0=15.639999999999999),
 Row(0=17.0),
 Row(0=12.76),
 Row(0=15.190000000000001),
 Row(0=19.980000000000004),
 Row(0=16.32),
 Row(0=14.399999999999999),
 Row(0=12.899999999999999),
 Row(0=23.2),
 Row(0=25.080000000000002),
 Row(0=21.060000000000002),
 Row(0=17.849999999999998),
 Row(0=21.66),
 Row(0=19.38),
 Row(0=18.36),
 Row(0=18.87),
 Row(0=16.56),
 Row(0=16.83),
 Row(0=16.32),
 Row(0=15.0),
 Row(0=17.0),
 Row(0=18.2),
 Row(0=17.68),
 Row(0=15.040000000000001),
 Row(0=14.879999999999999),
 Row(0=18.36),
 Row(0=21.32),
 Row(0=23.1),
 Row(0=15.190000000000001),
 Row(0=16.0),
 Row(0=19.25),
 Row(0=15.190000000000001),
 Row(0=13.200000000000001),
 Row(0=17.34),
 Row(0=17.5),
 Row(0=10.35),
 Row(0=14.080000000000002),
 Row(0=17.5),
 Row(0=19.38),
 Row(0=14.399999999999999),
 Row(0=19.38),
 Row(0=14.719999999999999),
 Row(0=19.61),
 Row(0=16.5),
 

In [32]:
coll_list = [i[0] for i in coll_cl]

In [33]:
coll_list

[17.849999999999998,
 14.700000000000001,
 15.040000000000001,
 14.26,
 18.0,
 21.060000000000002,
 15.639999999999999,
 17.0,
 12.76,
 15.190000000000001,
 19.980000000000004,
 16.32,
 14.399999999999999,
 12.899999999999999,
 23.2,
 25.080000000000002,
 21.060000000000002,
 17.849999999999998,
 21.66,
 19.38,
 18.36,
 18.87,
 16.56,
 16.83,
 16.32,
 15.0,
 17.0,
 18.2,
 17.68,
 15.040000000000001,
 14.879999999999999,
 18.36,
 21.32,
 23.1,
 15.190000000000001,
 16.0,
 19.25,
 15.190000000000001,
 13.200000000000001,
 17.34,
 17.5,
 10.35,
 14.080000000000002,
 17.5,
 19.38,
 14.399999999999999,
 19.38,
 14.719999999999999,
 19.61,
 16.5,
 22.400000000000002,
 20.480000000000004,
 21.39,
 12.649999999999999,
 18.2,
 15.959999999999999,
 20.79,
 11.76,
 19.139999999999997,
 14.040000000000001,
 10.0,
 17.700000000000003,
 13.200000000000001,
 17.689999999999998,
 16.24,
 20.77,
 16.799999999999997,
 15.66,
 13.640000000000002,
 14.0,
 18.880000000000003,
 17.08,
 15.75,
 17.08,
 18.56

In [34]:
lit(0)

Column<b'0'>

In [35]:
dataset.columns

['_c0', '_c1', '_c2', '_c3', '_c4', 'features', 'label']

In [36]:
dataset.select(['_c0', '_c1']).show(5)

+---+---+
|_c0|_c1|
+---+---+
|5.1|3.5|
|4.9|3.0|
|4.7|3.2|
|4.6|3.1|
|5.0|3.6|
+---+---+
only showing top 5 rows



In [37]:
dataset_delete = dataset.select([i for i in dataset.columns if i != 'label'])

In [38]:
dataset_delete.show(5)

+---+---+---+---+---+-----------------+
|_c0|_c1|_c2|_c3|_c4|         features|
+---+---+---+---+---+-----------------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|
+---+---+---+---+---+-----------------+
only showing top 5 rows



In [39]:
dataset._c0

Column<b'_c0'>

In [40]:
new_dataset = dataset.withColumn('_c00', dataset['_c0'] + 2)

In [41]:
new_dataset

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double, _c00: double]

In [42]:
new_dataset.show()

+---+---+---+---+---+-----------------+-----+----+
|_c0|_c1|_c2|_c3|_c4|         features|label|_c00|
+---+---+---+---+---+-----------------+-----+----+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  0.0| 7.1|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  0.0| 6.9|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  0.0| 6.7|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0| 6.6|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  0.0| 7.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  0.0| 7.4|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0| 6.6|
|5.0|3.4|1.5|0.2|1.0|[5.0,3.4,1.5,0.2]|  0.0| 7.0|
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0| 6.4|
|4.9|3.1|1.5|0.1|1.0|[4.9,3.1,1.5,0.1]|  0.0| 6.9|
|5.4|3.7|1.5|0.2|1.0|[5.4,3.7,1.5,0.2]|  0.0| 7.4|
|4.8|3.4|1.6|0.2|1.0|[4.8,3.4,1.6,0.2]|  0.0| 6.8|
|4.8|3.0|1.4|0.1|1.0|[4.8,3.0,1.4,0.1]|  0.0| 6.8|
|4.3|3.0|1.1|0.1|1.0|[4.3,3.0,1.1,0.1]|  0.0| 6.3|
|5.8|4.0|1.2|0.2|1.0|[5.8,4.0,1.2,0.2]|  0.0| 7.8|
|5.7|4.4|1.5|0.4|1.0|[5.7,4.4,1.5,0.4]|  0.0| 7.7|
|5.4|3.9|1.3|0.4|1.0|[5.4,3.9,1

In [43]:
type(dataset.select('_c0'))

pyspark.sql.dataframe.DataFrame

In [44]:
dataset_cl= dataset.select('_c0').collect()

In [45]:
dataset_cl

[Row(_c0=5.1),
 Row(_c0=4.9),
 Row(_c0=4.7),
 Row(_c0=4.6),
 Row(_c0=5.0),
 Row(_c0=5.4),
 Row(_c0=4.6),
 Row(_c0=5.0),
 Row(_c0=4.4),
 Row(_c0=4.9),
 Row(_c0=5.4),
 Row(_c0=4.8),
 Row(_c0=4.8),
 Row(_c0=4.3),
 Row(_c0=5.8),
 Row(_c0=5.7),
 Row(_c0=5.4),
 Row(_c0=5.1),
 Row(_c0=5.7),
 Row(_c0=5.1),
 Row(_c0=5.4),
 Row(_c0=5.1),
 Row(_c0=4.6),
 Row(_c0=5.1),
 Row(_c0=4.8),
 Row(_c0=5.0),
 Row(_c0=5.0),
 Row(_c0=5.2),
 Row(_c0=5.2),
 Row(_c0=4.7),
 Row(_c0=4.8),
 Row(_c0=5.4),
 Row(_c0=5.2),
 Row(_c0=5.5),
 Row(_c0=4.9),
 Row(_c0=5.0),
 Row(_c0=5.5),
 Row(_c0=4.9),
 Row(_c0=4.4),
 Row(_c0=5.1),
 Row(_c0=5.0),
 Row(_c0=4.5),
 Row(_c0=4.4),
 Row(_c0=5.0),
 Row(_c0=5.1),
 Row(_c0=4.8),
 Row(_c0=5.1),
 Row(_c0=4.6),
 Row(_c0=5.3),
 Row(_c0=5.0),
 Row(_c0=7.0),
 Row(_c0=6.4),
 Row(_c0=6.9),
 Row(_c0=5.5),
 Row(_c0=6.5),
 Row(_c0=5.7),
 Row(_c0=6.3),
 Row(_c0=4.9),
 Row(_c0=6.6),
 Row(_c0=5.2),
 Row(_c0=5.0),
 Row(_c0=5.9),
 Row(_c0=6.0),
 Row(_c0=6.1),
 Row(_c0=5.6),
 Row(_c0=6.7),
 Row(_c0=5

In [46]:
# dataset_list = [i[0] for i in dataset_cl]  # OK
dataset_list = [i['_c0'] for i in dataset_cl]  # OK

In [47]:
dataset_list

[5.1,
 4.9,
 4.7,
 4.6,
 5.0,
 5.4,
 4.6,
 5.0,
 4.4,
 4.9,
 5.4,
 4.8,
 4.8,
 4.3,
 5.8,
 5.7,
 5.4,
 5.1,
 5.7,
 5.1,
 5.4,
 5.1,
 4.6,
 5.1,
 4.8,
 5.0,
 5.0,
 5.2,
 5.2,
 4.7,
 4.8,
 5.4,
 5.2,
 5.5,
 4.9,
 5.0,
 5.5,
 4.9,
 4.4,
 5.1,
 5.0,
 4.5,
 4.4,
 5.0,
 5.1,
 4.8,
 5.1,
 4.6,
 5.3,
 5.0,
 7.0,
 6.4,
 6.9,
 5.5,
 6.5,
 5.7,
 6.3,
 4.9,
 6.6,
 5.2,
 5.0,
 5.9,
 6.0,
 6.1,
 5.6,
 6.7,
 5.6,
 5.8,
 6.2,
 5.6,
 5.9,
 6.1,
 6.3,
 6.1,
 6.4,
 6.6,
 6.8,
 6.7,
 6.0,
 5.7,
 5.5,
 5.5,
 5.8,
 6.0,
 5.4,
 6.0,
 6.7,
 6.3,
 5.6,
 5.5,
 5.5,
 6.1,
 5.8,
 5.0,
 5.6,
 5.7,
 5.7,
 6.2,
 5.1,
 5.7,
 6.3,
 5.8,
 7.1,
 6.3,
 6.5,
 7.6,
 4.9,
 7.3,
 6.7,
 7.2,
 6.5,
 6.4,
 6.8,
 5.7,
 5.8,
 6.4,
 6.5,
 7.7,
 7.7,
 6.0,
 6.9,
 5.6,
 7.7,
 6.3,
 6.7,
 7.2,
 6.2,
 6.1,
 6.4,
 7.2,
 7.4,
 7.9,
 6.4,
 6.3,
 6.1,
 7.7,
 6.3,
 6.4,
 6.0,
 6.9,
 6.7,
 6.9,
 5.8,
 6.8,
 6.7,
 6.7,
 6.3,
 6.5,
 6.2,
 5.9]

In [48]:
type(dataset_list[0])

float

In [49]:
# 删除某些无用列
new_dataset = dataset.drop('_c0', '_c1', '_c2', '_c3', '_c4')

In [50]:
new_dataset.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
+-----------------+-----+
only showing top 5 rows



In [51]:
# 新_模型训练
lr_2 = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel_2 = lr_2.fit(new_dataset)

In [52]:
lrModel_2.save('C:\Wang Hanmo\scripts\spark\lr')

Py4JJavaError: An error occurred while calling o368.save.
: java.io.IOException: Path C:\Wang Hanmo\scripts\spark\lr already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:503)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:102)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(C

In [53]:
lr_model = LogisticRegression.load('C:\Wang Hanmo\scripts\spark\lr')

Py4JJavaError: An error occurred while calling o384.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.classification.LogisticRegressionModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:333)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(

In [54]:
lr_model = LogisticRegression.load('C:\Wang Hanmo\scripts\spark\lr')

Py4JJavaError: An error occurred while calling o401.load.
: java.lang.NoSuchMethodException: org.apache.spark.ml.classification.LogisticRegressionModel.<init>(java.lang.String)
	at java.lang.Class.getConstructor0(Class.java:3082)
	at java.lang.Class.getConstructor(Class.java:1825)
	at org.apache.spark.ml.util.DefaultParamsReader.load(ReadWrite.scala:333)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(

In [55]:
lr_model = LogisticRegressionModel.load('C:\Wang Hanmo\scripts\spark\lr')

In [56]:
predictions4 = lr_model.transform(new_dataset, thresholds=0.6)

TypeError: transform() got an unexpected keyword argument 'thresholds'

In [ ]:
predictions4.show(5)

In [57]:
# 修改阈值
lr_model.params

[Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'),
 Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'),
 Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial.'),
 Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='featuresCol', doc='features column name'),
 Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='fitIntercept', doc='whether to fit an intercept term'),
 Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='labelCol', doc='label column name'),
 Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='lowerBoundsOnCoefficients', doc='The lowe

In [58]:
lr_model.getParam('threshold')

Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='threshold', doc='threshold in binary classification prediction, in range [0, 1]')

In [59]:
lr_model.getParam('thresholds')

Param(parent='LogisticRegression_4e52a129ba750364bdcf', name='thresholds', doc="Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0 excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold")

In [60]:
lr_model.isDefined('thresholds')

False

In [61]:
lr_model.isSet('thresholds')

False

In [62]:
lr_model_read = lr_model.read()

In [63]:
lr_model_read

In [64]:
lr_model_read.set('threshold', 0.6)

AttributeError: 'JavaMLReader' object has no attribute 'set'

In [65]:
lr_model.set(threshold, 0.6)

NameError: name 'threshold' is not defined

In [66]:
lr_model

LogisticRegression_4e52a129ba750364bdcf

In [67]:
prediction_3 = lr_model.transform(new_dataset)

In [68]:
prediction_3.show()

+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[5.1,3.5,1.4,0.2]|  0.0|[1.50553010141029...|[0.80168217364268...|       0.0|
|[4.9,3.0,1.4,0.2]|  0.0|[1.20141808462763...|[0.69508498886403...|       0.0|
|[4.7,3.2,1.3,0.2]|  0.0|[1.43534662127354...|[0.76732997443658...|       0.0|
|[4.6,3.1,1.5,0.2]|  0.0|[1.34214957417624...|[0.73710671332729...|       0.0|
|[5.0,3.6,1.4,0.2]|  0.0|[1.60975088260149...|[0.82670286305514...|       0.0|
|[5.4,3.9,1.7,0.4]|  0.0|[1.52181555631473...|[0.82246378795528...|       0.0|
|[4.6,3.4,1.4,0.3]|  0.0|[1.53372993913371...|[0.80131748475294...|       0.0|
|[5.0,3.4,1.5,0.2]|  0.0|[1.43782002857652...|[0.78118697494117...|       0.0|
|[4.4,2.9,1.4,0.2]|  0.0|[1.28319035129927...|[0.70504407252594...|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|[1.30272547919305...|[0.727

In [69]:
prediction_2 = lrModel_2.transform(new_dataset)

In [70]:
prediction_2.show(150)

+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[5.1,3.5,1.4,0.2]|  0.0|[1.50553010141029...|[0.80168217364268...|       0.0|
|[4.9,3.0,1.4,0.2]|  0.0|[1.20141808462763...|[0.69508498886403...|       0.0|
|[4.7,3.2,1.3,0.2]|  0.0|[1.43534662127354...|[0.76732997443658...|       0.0|
|[4.6,3.1,1.5,0.2]|  0.0|[1.34214957417624...|[0.73710671332729...|       0.0|
|[5.0,3.6,1.4,0.2]|  0.0|[1.60975088260149...|[0.82670286305514...|       0.0|
|[5.4,3.9,1.7,0.4]|  0.0|[1.52181555631473...|[0.82246378795528...|       0.0|
|[4.6,3.4,1.4,0.3]|  0.0|[1.53372993913371...|[0.80131748475294...|       0.0|
|[5.0,3.4,1.5,0.2]|  0.0|[1.43782002857652...|[0.78118697494117...|       0.0|
|[4.4,2.9,1.4,0.2]|  0.0|[1.28319035129927...|[0.70504407252594...|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|[1.30272547919305...|[0.727

In [71]:
label_list = prediction_2.select('label').collect()

In [72]:
prob_list = prediction_2.select('probability').collect()

In [73]:
type(label_list)

list

In [75]:
type(label_list[0][0])

float

In [76]:
label_list[0][0]

0.0

In [77]:
label_list = [label_list[i][0] for i in range(0, len(label_list))]

In [78]:
prob_list = prediction_2.select('probability').collect()

In [79]:
prob_list[0][0]

DenseVector([0.8017, 0.0461, 0.1522])

In [80]:
prob_list = [prob_list[i][0] for i in range(0, len(prob_list))]

In [81]:
prob_list


[DenseVector([0.8017, 0.0461, 0.1522]),
 DenseVector([0.6951, 0.052, 0.2529]),
 DenseVector([0.7673, 0.0415, 0.1911]),
 DenseVector([0.7371, 0.0443, 0.2185]),
 DenseVector([0.8267, 0.0414, 0.1319]),
 DenseVector([0.8225, 0.0598, 0.1178]),
 DenseVector([0.8013, 0.0408, 0.1578]),
 DenseVector([0.7812, 0.0478, 0.1711]),
 DenseVector([0.705, 0.0418, 0.2532]),
 DenseVector([0.7276, 0.0472, 0.2252]),
 DenseVector([0.8171, 0.0508, 0.1321]),
 DenseVector([0.7881, 0.0445, 0.1673]),
 DenseVector([0.7151, 0.045, 0.2398]),
 DenseVector([0.7684, 0.031, 0.2006]),
 DenseVector([0.8592, 0.0474, 0.0934]),
 DenseVector([0.8875, 0.0483, 0.0642]),
 DenseVector([0.8425, 0.0511, 0.1064]),
 DenseVector([0.7912, 0.0509, 0.1579]),
 DenseVector([0.7955, 0.0673, 0.1372]),
 DenseVector([0.842, 0.0449, 0.1131]),
 DenseVector([0.7408, 0.0635, 0.1957]),
 DenseVector([0.8148, 0.0524, 0.1327]),
 DenseVector([0.8644, 0.0282, 0.1073]),
 DenseVector([0.6994, 0.0748, 0.2257]),
 DenseVector([0.7704, 0.0498, 0.1797]),
 Dens

In [82]:
label_list[0] >= 0.0

True

In [83]:
for i in label_list:
    print(type(i))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class '

In [84]:
trainingSummary = lrModel_2.summary
trainingSummary

In [85]:
trainingSummary.falsePositiveRateByLabel

[0.0, 0.11, 0.05]

In [86]:
trainingSummary.accuracy

0.8933333333333333

In [87]:
trainingSummary.precisionByLabel    # 按照0,1,2

[1.0, 0.8035714285714286, 0.8863636363636364]

In [88]:
trainingSummary.labels

[0.0, 1.0, 2.0]

In [89]:
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

objectiveHistory:
1.098612288668108
1.0216272508645472
0.9098834909238206
0.7232617195708845
0.7134175932240653
0.71174618304454
0.7094361506077577
0.7051591389710494
0.7037641889472712
0.7028953520885549
0.7020092343060929
0.6971842530498781
0.6953614017395369
0.6843212905397233
0.6736512616220666
0.6661461231730135
0.6660124737236053
0.6646324729420506
0.6644255265375019
0.6643910980727231
0.6643372451822104


In [90]:
float(1)

1.0

In [91]:
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))


AttributeError: 'LogisticRegressionTrainingSummary' object has no attribute 'roc'

In [92]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], 123)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))
# trainingData.show(10)
testData.show(5)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|
|4.4|3.0|1.3|0.2|1.0|[4.4,3.0,1.3,0.2]|  0.0|
|4.4|3.2|1.3|0.2|1.0|[4.4,3.2,1.3,0.2]|  0.0|
|4.6|3.6|1.0|0.2|1.0|[4.6,3.6,1.0,0.2]|  0.0|
|4.7|3.2|1.6|0.2|1.0|[4.7,3.2,1.6,0.2]|  0.0|
+---+---+---+---+---+-----------------+-----+
only showing top 5 rows



In [93]:
trainingData.show(5)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|4.3|3.0|1.1|0.1|1.0|[4.3,3.0,1.1,0.1]|  0.0|
|4.5|2.3|1.3|0.3|1.0|[4.5,2.3,1.3,0.3]|  0.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0|
|4.6|3.2|1.4|0.2|1.0|[4.6,3.2,1.4,0.2]|  0.0|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0|
+---+---+---+---+---+-----------------+-----+
only showing top 5 rows



In [94]:
# 模型训练
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

In [95]:
# 模型预测
predictions = lrModel.transform(testData)
# print(prediction)
predictions.show(10)

# ROC score
# evaluator = BinaryClassificationEvaluator()
# evaluator.evaluate(prediction)

+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|_c0|_c1|_c2|_c3|_c4|         features|label|       rawPrediction|         probability|prediction|
+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|[0.97587696525420...|[0.66435501880071...|       0.0|
|4.4|3.0|1.3|0.2|1.0|[4.4,3.0,1.3,0.2]|  0.0|[1.07468573243426...|[0.69738164615681...|       0.0|
|4.4|3.2|1.3|0.2|1.0|[4.4,3.2,1.3,0.2]|  0.0|[1.22655912321016...|[0.74546466374773...|       0.0|
|4.6|3.6|1.0|0.2|1.0|[4.6,3.6,1.0,0.2]|  0.0|[1.55117580808555...|[0.82793089816996...|       0.0|
|4.7|3.2|1.6|0.2|1.0|[4.7,3.2,1.6,0.2]|  0.0|[1.08632343975477...|[0.70280652702730...|       0.0|
|4.9|2.4|3.3|1.0|2.0|[4.9,2.4,3.3,1.0]|  2.0|[-0.3580763884417...|[0.22319947512485...|       2.0|
|5.0|3.0|1.6|0.2|1.0|[5.0,3.0,1.6,0.2]|  0.0|[0.86283058089978...|[0.62295572416110...|       0.0|
|5.0|3.4|1

In [96]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [97]:
accuracy

0.8857142857142857

In [98]:
preSummary = prediction.summary
preSummary

NameError: name 'prediction' is not defined

In [ ]:
preSummary.accuracy

In [ ]:
preSummary.weightedRecall